In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')

print("Train DataFrame Shape:", train_df.shape)
print("Test DataFrame Shape:", test_df.shape)


Train DataFrame Shape: (891, 12)
Test DataFrame Shape: (418, 11)


In [10]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Deck
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Q,M
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,S,M
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,Q,M
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,S,M
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,S,M


In [3]:
train_df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [4]:
test_df.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [5]:
def preprocess_data_improved(train_df, test_df):
    
    # 1. Calculate Imputation Values ONLY from the Training Set
    # Age: Use mean
    train_mean_age = train_df['Age'].mean()
    # Embarked: Use mode
    train_mode_embarked = train_df['Embarked'].mode()[0]
    # Fare: Use median for robustness against outliers (from train_df or test_df as needed)
    test_median_fare = test_df['Fare'].median() # Only need to calculate for test since train has 0 nulls

    # 2. Impute Embarked (using train mode for both)
    train_df['Embarked'].fillna(train_mode_embarked, inplace=True)
    test_df['Embarked'].fillna(train_mode_embarked, inplace=True) # Apply train mode to test set too (safer)

    # 3. Impute Fare (using test median for test set)
    test_df['Fare'].fillna(test_median_fare, inplace=True)

    # 4. Impute Age (using train mean for both)
    train_df['Age'].fillna(train_mean_age, inplace=True)
    test_df['Age'].fillna(train_mean_age, inplace=True)

    # 5. Cabin/Deck and Title Feature Engineering
    for df in [train_df, test_df]:
    # Cabin/Deck Feature
        df['Deck'] = df['Cabin'].str[0].fillna('M')
        df.drop('Cabin', axis=1, inplace=True)
    
    # Title Feature (using robust RAW string regex extraction)
        df['Title'] = df['Name'].str.extract(r' ([A-Za-z]+)\.', expand=False) # <--- FIXED LINE
    
    # Simplify/Group Titles
        df['Title'] = df['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona', 'Mlle', 'Ms', 'Mme'], 
                                      ['Rare', 'Rare', 'Rare', 'Rare', 'Rare', 'Rare', 'Rare', 'Rare', 'Rare', 'Rare', 'Rare', 'Miss', 'Miss', 'Mrs'])
    
    # Drop the original Name column
        df.drop('Name', axis=1, inplace=True)
        
    return train_df, test_df

train_df, test_df = preprocess_data_improved(train_df, test_df)

/var/folders/ff/x3h3wrpn2hl03b0nwy1n8xk00000gn/T/ipykernel_71981/3687815108.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df['Embarked'].fillna(train_mode_embarked, inplace=True)
/var/folders/ff/x3h3wrpn2hl03b0nwy1n8xk00000gn/T/ipykernel_71981/3687815108.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are set

In [6]:
print(train_df.isnull().sum())
print(test_df.isnull().sum())

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
Deck           0
Title          0
dtype: int64
PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
Deck           0
Title          0
dtype: int64


In [8]:
train_df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Deck,Title
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,S,M,Mr
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C,C,Mrs
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,S,M,Miss
3,4,1,1,female,35.0,1,0,113803,53.1000,S,C,Mrs
4,5,0,3,male,35.0,0,0,373450,8.0500,S,M,Mr
